### This time it works !!!

In [56]:
import pandas as pd
import random
from numba import njit

In [57]:
data = {
        'Age': list(range(10000)),
        'Height' : list(range(10000))
        }

df = pd.DataFrame(data)

In [68]:
df["Age"].sum()

49995000

In [69]:
def get_age_sum(df):
    return df["Age"].sum()

In [73]:
get_age_sum(df)

49995000

In [74]:
get_age_sum_jitted = njit(get_age_sum)

In [75]:
get_age_sum_jitted(df)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
non-precise type pyobject
During: typing of argument at /var/folders/wh/73lk8tdn5qq9g7nhp1hmg01m0000gn/T/ipykernel_76801/3754958239.py (2)

File "../../../../../var/folders/wh/73lk8tdn5qq9g7nhp1hmg01m0000gn/T/ipykernel_76801/3754958239.py", line 2:
<source missing, REPL/exec in use?> 

This error may have been caused by the following argument(s):
- argument 0: Cannot determine Numba type of <class 'pandas.core.frame.DataFrame'>


### This time, it really works !!

In [76]:
data = pd.Series(range(1_000_000))

roll = data.rolling(10)

def f(x):
    return np.sum(x) + 5

In [86]:
roll.apply(f, raw=True)

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
            ...    
999995    9999910.0
999996    9999920.0
999997    9999930.0
999998    9999940.0
999999    9999950.0
Length: 1000000, dtype: float64

In [ ]:
import numba


@numba.jit
def f_plain(x):
    return x * (x - 1)


@numba.jit
def integrate_f_numba(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_plain(a + i * dx)
    return s * dx


@numba.jit
def apply_integrate_f_numba(col_a, col_b, col_N):
    n = len(col_N)
    result = np.empty(n, dtype="float64")
    assert len(col_a) == len(col_b) == n
    for i in range(n):
        result[i] = integrate_f_numba(col_a[i], col_b[i], col_N[i])
    return result


def compute_numba(df):
    result = apply_integrate_f_numba(
        df["a"].to_numpy(), df["b"].to_numpy(), df["N"].to_numpy()
    )
    return pd.Series(result, index=df.index, name="result")

In [ ]:
compute_numba(df)